In [1]:
import pandas as pd
import random
import gc
import datatable

ModuleNotFoundError: No module named 'datatable'

In [2]:
random.seed(1)

In [3]:
train = pd.read_csv('./input/riiid-test-answer-prediction/train.csv',
                   dtype={'row_id': 'int64',
                          'timestamp': 'int64',
                          'user_id': 'int32',
                          'content_id': 'int16',
                          'content_type_id': 'int8',
                          'task_container_id': 'int16',
                          'user_answer': 'int8',
                          'answered_correctly':'int8',
                          'prior_question_elapsed_time': 'float32',
                          'prior_question_had_explanation': 'boolean'}
                   )

Use the last several entry for each user as validation set sorted by timestamp. Spliting may focus on light users more.

In [4]:
valid_split1 = train.groupby('user_id').tail(5)
train_split1 = train[~train.row_id.isin(valid_split1.row_id)]
valid_split = valid_split1[valid_split1.content_type_id == 0]
train_split = train_split1[train_split1.content_type_id == 0]
valid_split_w_lecture = valid_split1[valid_split1.content_type_id == 1]
train_split_w_lecture = train_split1[train_split1.content_type_id == 1]
print("{:.3}  {:.3}".format(train_split.answered_correctly.mean(), valid_split.answered_correctly.mean()))

0.66  0.541


Since training data and test data are split by time, the validation data should also be split by time. However, the given timestamp is the time that has elapsed since the user's first event, not the actual time, so I set a random first access time for each user within a certain interval.

In [5]:
max_timestamp_u = train[['user_id','timestamp']].groupby(['user_id']).agg(['max']).reset_index()
max_timestamp_u.columns = ['user_id', 'max_time_stamp']
MAX_TIME_STAMP = max_timestamp_u.max_time_stamp.max()

def rand_time(max_time_stamp):
    interval = MAX_TIME_STAMP - max_time_stamp
    rand_time_stamp = random.randint(0,interval)
    return rand_time_stamp

max_timestamp_u['rand_time_stamp'] = max_timestamp_u.max_time_stamp.apply(rand_time)
train = train.merge(max_timestamp_u, on='user_id', how='left')
train['virtual_time_stamp'] = train.timestamp + train['rand_time_stamp']

In [6]:
max_timestamp_u

,user_id,max_time_stamp,rand_time_stamp
0,115,668090043,77886501365
1,124,571323,11870321604
2,2746,835457,13980415036
3,5382,2101551456,66355058851
4,8623,862338736,54338178075
...,...,...,...
393651,2147470770,2832089444,43282978945
393652,2147470777,13332685203,69123704590
393653,2147481750,55954768,45075041930
393654,2147482216,1470594073,80935632229


In [7]:
train = train.sort_values(['virtual_time_stamp', 'row_id']).reset_index(drop=True)

In [8]:
val_size = 2500000

for cv in range(5):
    valid = train[-val_size:]
    train = train[:-val_size]
    # check new users and new contents
    new_users = len(valid[~valid.user_id.isin(train.user_id)].user_id.unique())
    valid_question = valid[valid.content_type_id == 0]
    train_question = train[train.content_type_id == 0]
    new_contents = len(valid_question[~valid_question.content_id.isin(train_question.content_id)].content_id.unique())    
    print(f'cv{cv} {train_question.answered_correctly.mean():.3f} {valid_question.answered_correctly.mean():.3f} {new_users} {new_contents}')
    valid.to_pickle(f'cv{cv+1}_valid.pickle')
    train.to_pickle(f'cv{cv+1}_train.pickle')

cv0 0.658 0.642 15119 0
cv1 0.658 0.651 11198 0
cv2 0.658 0.647 10159 0
cv3 0.658 0.651 9687 3
cv4 0.658 0.655 9184 0
